In [1]:

import numpy as np
import os
import pandas as pd


def get_input_csv(day: int):
    fp = os.path.join(os.getcwd(), "inputs", f"day{day}.csv")
    return pd.read_csv(fp, encoding="utf-8", header=None).rename(columns={0: "data"})


def get_input_text(day: int):
    fp = os.path.join(os.getcwd(), "inputs", f"day{day}.txt")
    with open(fp, "r") as f:
        return [s.replace("\n", "") for s in f.readlines()]

## Day 1

In [2]:
# Input:
data = get_input_csv(1)
data = list(data["data"].values)
data[:10]

[151, 152, 153, 158, 159, 163, 164, 162, 161, 167]

In [3]:
# Q1 solution:
answer = len(list(filter(lambda x: x[1] > data[x[0]], list(enumerate(data[1:])))))
answer

1564

In [4]:
# Q2 solution:
answer = len(list(filter(lambda ix: sum(data[ix-3: ix]) < sum(data[ix-2: ix+1]), range(3, len(data), 1))))
answer

1611

## Day 2

In [5]:
# Input:
data = get_input_csv(2)
data = list(data["data"].values)
data[:10]

['forward 3',
 'down 6',
 'forward 7',
 'down 4',
 'down 8',
 'down 6',
 'down 4',
 'down 6',
 'forward 7',
 'down 5']

In [6]:
# Q1 solution - needlessly compact 3 liner:
h, d = 0, 0
[exec({"f": "global h;h= h+int(i)", "d": "global d;d+=int(i)", "u": "global d;d-=int(i)"}[dr[0]]) for dr, i in map(str.split, data)]
answer = h*d
answer

1924923

In [7]:
# Q1 solution - slightly less needlessly compact 4 liner:
h, d = 0, 0
for dct in [{"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]] for s, i in map(str.split, data)]:
    h += dct.get("h", 0)
    d += dct.get("d", 0)
answer = h*d
answer

1924923

In [8]:
# Q1 solution - reduce:
from functools import reduce

iterable = [{"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]] for s, i in map(str.split, data)]
d = reduce(lambda x, y: x + y.get("d", 0), iterable, 0)
h = reduce(lambda x, y: x + y.get("h", 0), iterable, 0)
answer = h*d
answer

1924923

In [9]:
# Q1 solution - recursion:
def recursion(seq, h=0, d=0):
    if len(seq):
        s, i = seq[0].split()
        dct = {"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]]
        h += dct.get("h", 0)
        d += dct.get("d", 0)
        return recursion(seq[1:], h, d)
    else:
        return h * d
        
        
answer = recursion(data)
answer

1924923

In [10]:
# Q2 solution - basic brute force:
h, d, a = 0, 0, 0
for s, i in map(str.split, data):
    if s[0] == "f":
        h += int(i)
        d += a*int(i)
    elif s[0] == "d":
        a += int(i)
    elif s[0] == "u":
        a -= int(i)

answer = h*d
answer

1982495697

## Day 3

In [11]:
# Input:
data = get_input_text(3)
data[:10]

['000100011010',
 '110011110110',
 '011000101111',
 '001101100101',
 '011100001000',
 '101101011011',
 '101111010101',
 '011010000101',
 '010101000010',
 '100001111000']

In [12]:
# Q1 - Basic solution:
zeroes_ones = [[0, 0] for s in range(len(data[0]))]
for s in data:
    for i, num in enumerate(s):
        if num == "0":
            zeroes_ones[i][0] += 1
        else:
            zeroes_ones[i][1] += 1

gamma = "".join([str(int(max(l) == l[1])) for l in zeroes_ones])
epsilon = "".join(["1" if i =="0" else "0" for i in gamma])

answer = int(gamma, 2) * int(epsilon, 2)
answer

2595824

In [13]:
# Q2 - Basic solution:

ox, co2 = None, None
oxs, co2s = data.copy(), data.copy()

for pos in range(len(data[0])):
    
    if ox is None:
        ox_zero_count = len(list(filter(lambda s: s[pos] == "0", oxs)))
        ox_one_count = len(oxs) - ox_zero_count
        ox_num = int(ox_one_count >= ox_zero_count)
        oxs = list(filter(lambda s: s[pos] == str(ox_num), oxs))
        if len(oxs) == 1:
            ox = int(oxs[0], 2)
    
    if co2 is None:
        co2_zero_count = len(list(filter(lambda s: s[pos] == "0", co2s)))
        co2_one_count = len(co2s) - co2_zero_count
        co2_num = int(co2_zero_count > co2_one_count)
        co2s = list(filter(lambda s: s[pos] == str(co2_num), co2s))
        if len(co2s) == 1:
            co2 = int(co2s[0], 2)

answer = ox * co2
answer

2135254

## Day 4

In [14]:
# Parse data:
fp = os.path.join(os.getcwd(), "inputs", f"day4.txt")
with open(fp, "r") as f:
    data = f.readlines()
order = [int(i) for i in data[0].split(",")]

boards = list()
rows = list()
for line in data[1:]:
    if len(rows) == 5:
        boards.append(pd.DataFrame(rows))
    if line == "\n":
        rows = list()
    else:
        rows.append([int(i) for i in line.split()])


In [15]:
# Q1 - brute force solution:
bingo = False
for last in range(5, len(order), 1):
    nums = order[:last]
    for board in boards:
        on_board = [1 if i in nums else 0 for i in board.values.flatten()]
        marked = pd.DataFrame(np.array(on_board).reshape(5, 5))
        bingo = (5 in marked.sum(axis=1).values) or (5 in marked.sum(axis=0).values)
        if bingo:
            mask = marked.astype(bool)
            unmarked_sum = board[~mask].sum().sum()
            break
    if bingo:
        break

answer = int(unmarked_sum * order[last-1])
answer

25410

In [16]:
# Q2 - brute force solution:
bingo = False
boards_completed = list()
last_called = -1
for last in range(5, len(order), 1):
    nums = order[:last]
    for i, board in enumerate(boards):
        if i not in boards_completed:
            on_board = [1 if i in nums else 0 for i in board.values.flatten()]
            marked = pd.DataFrame(np.array(on_board).reshape(5, 5))
            bingo = (5 in marked.sum(axis=1).values) or (5 in marked.sum(axis=0).values)
            if bingo:
                mask = marked.astype(bool)
                unmarked_sum = board[~mask].sum().sum()
                boards_completed.append(i)
    if len(boards_completed) == len(boards):
        break

answer = int(unmarked_sum * nums[-1])
answer

2730

## Day 5

In [17]:
# Full data:
data = get_input_text(5)
data[:10]

# # Sample:
# data = """0,9 -> 5,9
# 8,0 -> 0,8
# 9,4 -> 3,4
# 2,2 -> 2,1
# 7,0 -> 7,4
# 6,4 -> 2,0
# 0,9 -> 2,9
# 3,4 -> 1,4
# 0,0 -> 8,8
# 5,5 -> 8,2""".split("\n")

x1s = [int(s.split(",")[0]) for s in data]
y1s = [int(s.split(",")[1].split()[0]) for s in data]
x2s = [int(s.split("->")[1].split(",")[0]) for s in data]
y2s = [int(s.split("->")[1].split(",")[1]) for s in data]

In [18]:
# Q1 - pandas solution:
xmax = max(x1s+x2s)
ymax = max(y1s+y2s)
df = pd.DataFrame(0, index=range(ymax+1), columns=range(xmax+1))
for (x1, y1, x2, y2) in zip(x1s, y1s, x2s, y2s):
    if x1 == x2 or y1 == y2:
        x1, x2 = sorted([x1, x2])
        y1, y2 = sorted([y1, y2])
        df.iloc[y1:y2+1, x1:x2+1] = df.iloc[y1:y2+1, x1:x2+1].values + 1
values = df.stack()
answer = len(values[values > 1])
answer

7473

In [19]:
# Q2 brute force solution:
xmax = max(x1s+x2s)
ymax = max(y1s+y2s)
df = pd.DataFrame(0, index=range(ymax+1), columns=range(xmax+1))
for (x1, y1, x2, y2) in zip(x1s, y1s, x2s, y2s):
    if x1 < x2:
        x_step = 1
    elif x1 > x2:
        x_step = -1
    else:
        x_step = 0
    if y1 < y2:
        y_step = 1
    elif y1 > y2:
        y_step = -1
    else:
        y_step = 0
    
    drawn = False
    while not drawn:
        df.iloc[y1, x1] = df.iloc[y1, x1] + 1
        if (x1 == x2) and (y1 == y2):
            drawn = True
        x1 += x_step
        y1 += y_step
        
values = df.stack()
answer = len(values[values > 1])
answer

24164

## Day 6

In [20]:
data = get_input_text(6)
data = [int(i) for i in data[0].split(",")]

# Sample:
# data = [3,4,3,1,2]

In [21]:
# Q1 - brute force solution:

n_days = 80
fishes = data.copy()
for day in range(n_days):
    new_fish = list()
    for i, fish in enumerate(fishes):
        if fish == 0:
            fishes[i] = 6
            new_fish.append(8)
        else:
            fishes[i] = fish - 1
    fishes += new_fish

answer = len(fishes)
answer

350149

In [22]:
# Q2 recursive counter solution:

def recursive_solution(counts: dict, n_days: int):
    if n_days == 0:
        return sum(counts.values())
    next_day_counts = {i: 0 for i in range(9)}
    for k, v in counts.items():
        if k == 0:
            next_day_counts[8] = v
            next_day_counts[6] = next_day_counts.get(6, 0) + v
        else:
            next_day_counts[k - 1] = next_day_counts.get(k - 1, 0) + v
    return recursive_solution(next_day_counts, n_days-1)


from collections import Counter
fishes = data.copy()
counts = dict(Counter(fishes))
recursive_solution(counts, 256)

1590327954513

# Day 7

In [23]:
data = get_input_text(7)
data = [int(i) for i in data[0].split(",")]

# Sample:
# data = [16,1,2,0,4,2,7,1,2,14]

In [24]:
# Q1 brute force solution:
best_position, min_fuel = -1, float("inf")
for position in set(data):
    fuel_cost = 0
    for crab in data:
        fuel_cost += abs(crab - position)
    if fuel_cost < min_fuel:
        min_fuel = fuel_cost
        best_position = position
        
min_fuel

364898

In [25]:
# Q2 triangle number solution:
best_position, min_fuel = -1, float("inf")
for position in range(min(data), max(data) + 1):
    fuel_cost = 0
    for crab in [i for i in data if i != position]:
        distance = abs(crab - position)
        fuel_cost += (distance * (distance+ 1) / 2)
    if fuel_cost < min_fuel:
        min_fuel = int(fuel_cost)
        best_position = position
        
min_fuel

104149091

# Day 8

In [26]:
data = get_input_text(8)
# data = [int(i) for i in data[0].split(",")]

# Sample:
# data = [
#     "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe",
#     "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc", 
#     "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg",
#     "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb",
#     "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea",
#     "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb",
#     "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe",
#     "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef",
#     "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb",
#     "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce",
# ]

input_values = [d.split(" | ")[0] for d in data]
output_values = [d.split(" | ")[1] for d in data]

In [27]:
# Q1:
lengths = [2, 3, 4, 7]
answer = 0
for line in output_values:
    answer += len(list(filter(lambda x: len(x) in lengths, line.split())))
answer

318

In [28]:
def deduce_mapping(input_line):
    nums = input_line.split()
    mapping = {l: None for l in "abcdefg"}
    
    # The numbers we know in the input:
    one = [l for l in nums if len(l) == 2][0]
    four = [l for l in nums if len(l) == 4][0]
    seven = [l for l in nums if len(l) == 3][0]
    eight = [l for l in nums if len(l) == 7][0]
    
    # Subtracting one from seven gives us "a":
    mapping["a"] = "".join(set(seven) - set(one))
    
    # Determine 9:
    # Only 0/6/9 have 1 line missing:
    zero_six_nine = [n for n in nums if len(n) == 6]
    # Subtract four plus "a" from eight to get e or g:
    e_or_g = set(eight) - set(four + mapping["a"])
    # Only 9 won't have e and g:
    nine = [n for n in zero_six_nine if len(list(set(n) & e_or_g)) == 1][0]

    # Subtract nine from eight to get "e":
    mapping["e"] = "".join(set(eight) - set(nine))
    
    # Nine minus 4 gives us "a" and "g", and we already know "a":
    a_and_g = set(nine) - set(four)
    mapping["g"] = "".join(a_and_g - {mapping["a"]})
    
    # Intersection of one and zero/six will get us each:
    zero_six = [n for n in zero_six_nine if n != nine]
    zero = [n for n in zero_six if len(list(set(one) & set(n))) == 2][0]
    six = [n for n in zero_six if n != zero][0]
    
    # Get "c" from eight minus six:
    mapping["c"] = "".join(set(eight) - set(six))
    
    # Get "d" from eight minus zero:
    mapping["d"] = "".join(set(eight) - set(zero))
    
    # Get "b" from four minus one minus "d":
    mapping["b"] = "".join(set(four) - set(one) - {mapping["d"]})
    
    # "f" is last unused letter:
    mapping["f"] = "".join(set("abcdefg") - set(mapping.values()))
    
    # Invert mapping to get true characters:
    return {v: k for k, v in mapping.items()}


true_numbers = {
    "abcefg": 0, "cf": 1, "acdeg": 2, "acdfg": 3, "bcdf": 4, 
    "abdfg": 5, "abdefg": 6, "acf": 7, "abcdefg": 8, "abcdfg": 9
}
all_numbers = []
for line in data:
    input_line, output_line = line.split(" | ")
    mapping = deduce_mapping(input_line)
    number = ""
    for num in output_line.split():
        digit = true_numbers["".join(sorted([mapping[l] for l in num]))]
        number += str(digit)
    all_numbers.append(int(number))

answer = sum(all_numbers)
answer

996280

# Day 9

In [71]:
data = get_input_text(9)

# Sample:
data = [
    "2199943210",
    "3987894921",
    "9856789892",
    "8767896789",
    "9899965678",
]

In [72]:
# Q1 - brute force solution:
height_map = [[int(i) for i in row] for row in data]

lows = list()
n_rows = len(height_map)
n_cols = len(height_map[0])
for r_ix, row in enumerate(height_map):
    for c_ix, v in enumerate(row):
        adjacent = list()
        # Above:
        if r_ix > 0:
            adjacent.append(height_map[r_ix-1][c_ix])
        # Left:
        if c_ix > 0:
            adjacent.append(height_map[r_ix][c_ix-1])
        # Right:
        if c_ix < n_cols - 1:
            adjacent.append(height_map[r_ix][c_ix+1])
        # Below:
        if r_ix < n_rows - 1:
            adjacent.append(height_map[r_ix+1][c_ix])
        if all([i > v for i in adjacent]):
            lows.append((r_ix, c_ix, v))

answer = sum([l[2]+1 for l in lows])
answer

15

In [ ]:
# Q2 recursive basin expansion:

def get_adjacent(row_ix: int, col_ix: int):
    """Return indices and values of all adjacent neighbors."""
    adjacent = list()
    if row_ix > 0:
        adjacent.append((row_ix-1, col_ix, height_map[row_ix-1][col_ix]))
    # Left:
    if col_ix > 0:
        adjacent.append((row_ix, col_ix-1, height_map[row_ix][col_ix-1]))
    # Right:
    if col_ix < n_cols - 1:
        adjacent.append((row_ix, col_ix+1, height_map[row_ix][col_ix+1]))
    # Below:
    if row_ix < n_rows - 1:
        adjacent.append((row_ix+1, col_ix, height_map[row_ix+1][col_ix]))
    return adjacent


def get_basin(row_ix: int, col_ix: int, low: int, basin: set, checked: set):
    neighbors = get_adjacent(row_ix, col_ix)
    neighbors_in_basin = [i for i in neighbors if (i[2] != 9) and (i[2] >= low)]
    basin = basin | set(neighbors_in_basin)
    checked = checked | {(row_ix, col_ix, low)}
    to_check = set(neighbors_in_basin) - checked
    if not to_check:
        return basin
    else:
        for r, c, _ in to_check:
            return get_basin(r, c, low, basin, checked)
#     return basin, to_check

get_basin(*lows[0], {lows[0]}, {lows[0]})
#NOT FINISHED

In [75]:
lows

[(0, 1, 1), (0, 9, 0), (2, 2, 5), (4, 6, 5)]